In [40]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install html5lib 
!pip install lxml
!pip install plotly
!pip install yfinance
!pip install nbformat



In [43]:
import yfinance as yf
import pandas as pd
import json 
import requests
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from bs4 import BeautifulSoup
import plotly.io as pio
pio.renderers.default = "iframe"
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [52]:
# Extracting Tesla Stock Data Using yfinance 
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
print(tesla_data.head())

                               Open      High       Low     Close     Volume  \
Date                                                                           
2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667  281494500   
2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667  257806500   
2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000  123282000   
2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   77097000   
2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000  103003500   

                           Dividends  Stock Splits  
Date                                                
2010-06-29 00:00:00-04:00        0.0           0.0  
2010-06-30 00:00:00-04:00        0.0           0.0  
2010-07-01 00:00:00-04:00        0.0           0.0  
2010-07-02 00:00:00-04:00        0.0           0.0  
2010-07-06 00:00:00-04:00        0.0           0.0  


In [53]:
# Extracting Tesla Revenue Data Using Webscraping
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
response = requests.get(url)
html_data = response.text
soup = BeautifulSoup(html_data, "html.parser")
table = soup.find_all("table")[0]
# print(table)
rows = table.find_all("tr")
# print(rows)
data = []
for row in rows:
    cols = row.find_all("td")
    if len(cols) > 1: 
        revenue = cols[1].text.strip() 
        data.append([revenue])
tesla_revenue_df = pd.DataFrame(data, columns=["Revenue"])
print(tesla_revenue_df.head())

   Revenue
0  $53,823
1  $31,536
2  $24,578
3  $21,461
4  $11,759


In [54]:
# Extracting GameStop Stock Data Using yfinance
gamestop = yf.Ticker("GME")
gamestop_data = gamestop.history(period="max")
print(gamestop_data.head())


                               Open      High       Low     Close    Volume  \
Date                                                                          
2002-02-13 00:00:00-05:00  1.620129  1.693350  1.603296  1.691667  76216000   
2002-02-14 00:00:00-05:00  1.712707  1.716074  1.670626  1.683250  11021600   
2002-02-15 00:00:00-05:00  1.683250  1.687458  1.658001  1.674834   8389600   
2002-02-19 00:00:00-05:00  1.666418  1.666418  1.578047  1.607504   7410400   
2002-02-20 00:00:00-05:00  1.615920  1.662210  1.603296  1.662210   6892800   

                           Dividends  Stock Splits  
Date                                                
2002-02-13 00:00:00-05:00        0.0           0.0  
2002-02-14 00:00:00-05:00        0.0           0.0  
2002-02-15 00:00:00-05:00        0.0           0.0  
2002-02-19 00:00:00-05:00        0.0           0.0  
2002-02-20 00:00:00-05:00        0.0           0.0  


In [55]:
# Extracting GameStop Revenue Data Using Webscraping
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
response = requests.get(url)
html_data_2 = response.text
soup = BeautifulSoup(html_data_2, "html.parser")
tables = pd.read_html(str(soup))
gme_revenue = tables[1]
revenue_column = None
for col in gme_revenue.columns:
    if gme_revenue[col].str.contains(r'\$', na=False).any():  # Check if it contains '$'
        revenue_column = col
        break
if revenue_column:
    print(f"Data of the revenue column '{revenue_column}':")
    gme_revenue[revenue_column] = gme_revenue[revenue_column].replace({'\$': '', ',': ''}, regex=True).astype(float)
    print(gme_revenue[revenue_column].head())
else:
    print("Revenue column not found.")

Data of the revenue column 'GameStop Quarterly Revenue (Millions of US $).1':
0    1021.0
1    2194.0
2    1439.0
3    1286.0
4    1548.0
Name: GameStop Quarterly Revenue (Millions of US $).1, dtype: float64


In [49]:
# Function to create the dashboard
tesla_revenue_df['Revenue'] = tesla_revenue_df['Revenue'].replace({'\$': '', ',': ''}, regex=True).astype(float)

# Function to create the dashboard
def make_graph(stock_data, revenue_data, stock):
    # Create subplots for stock and revenue
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, subplot_titles=("Tesla Historical Share Price", "Tesla Historical Revenue"), vertical_spacing=0.3
    )
    
    # Add stock price trace (based on the index position)
    fig.add_trace(go.Scatter(
        x=stock_data.index,
        y=stock_data['Close'].astype("float"),
        name="Share Price"
    ), row=1, col=1)
    
    # Add revenue trace (based on index position, without using Date column)
    fig.add_trace(go.Scatter(
        x=revenue_data.index,
        y=revenue_data['Revenue'],
        name="Revenue"
    ), row=2, col=1)

    # Update axis labels
    fig.update_xaxes(title_text="Index", row=1, col=1)
    fig.update_xaxes(title_text="Index", row=2, col=1)
    fig.update_yaxes(title_text="Price (USD)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue (USD Millions)", row=2, col=1)
    
    # Update layout and display settings
    fig.update_layout(
        showlegend=False,
        height=900,
        title=f"{stock} Stock and Revenue Dashboard",
        xaxis_rangeslider_visible=True
    )
    
    # Display the plot
    fig.show()

# Create the Tesla Stock and Revenue Dashboard
make_graph(tesla_data, tesla_revenue_df, "Tesla")

In [51]:
# Function to create the GameStop Stock and Revenue Dashboard
def make_graph(stock_data, revenue_data, stock):
    # Create subplots for stock and revenue
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, subplot_titles=(f"{stock} Historical Share Price", f"{stock} Historical Revenue"), vertical_spacing=0.3
    )
    
    # Add stock price trace (based on the index position)
    fig.add_trace(go.Scatter(
        x=stock_data.index,
        y=stock_data['Close'].astype("float"),
        name="Share Price"
    ), row=1, col=1)
    
    # Add revenue trace (based on index position, without using Date column)
    fig.add_trace(go.Scatter(
        x=revenue_data.index,
        y=revenue_data[revenue_column],
        name="Revenue"
    ), row=2, col=1)

    # Update axis labels
    fig.update_xaxes(title_text="Index", row=1, col=1)
    fig.update_xaxes(title_text="Index", row=2, col=1)
    fig.update_yaxes(title_text="Price (USD)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue (USD Millions)", row=2, col=1)
    
    # Update layout and display settings
    fig.update_layout(
        showlegend=False,
        height=900,
        title=f"{stock} Stock and Revenue Dashboard",
        xaxis_rangeslider_visible=True
    )
    
    # Display the plot
    fig.show()

# Create the GameStop Stock and Revenue Dashboard
make_graph(gamestop_data, gme_revenue, "GameStop")